# MongoDB安装-linux

### 环境

* CentOS7
* MongoDB3.6.4

### 说明:

* mongodb下载解压安装最好切到root用户下进行

### 下载安装

到[官方网站](https://www.mongodb.com/download-center?jmp=nav#community)下载对应的包到一个目录,如/opt目录下

wget -o  https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-amazon-3.6.4.tgz 

解压文件到/opt/mongodb目录下： 

mkdir -p /opt/mongodb
tar -zxvf mongodb-linux-x86_64-3.6.4.tgz
cp -R -n mongodb-linux-x86_64-3.6.4/ /opt/mongodb

### 配置

#### 配置环境变量

设置mongodb环境变量，将环境变量放在用户主目录下的.bash_profile，在文件末尾添加

```bash
# export mongo path
export MONGODB_HOME=/opt/mongodb/bin
export PATH=$MONGODB_HOME:$PATH
export PATH
```

### 创建目录

```bash
cd /opt/mongodb
# 创建db目录
mkdir -p db
# 日志目录
mkdir -p logs
# pid目录
mkdir -p pid
```

### 创建配置文件

进入到bin目录下，创建mongodb.cfg文件

```bash
cd /opt/mongodb/bin
touch mongodb.cfg
vim mongodb.cfg
```

编辑mongodb.cfg文件, 以下是配置文件内容

### 测试

#### 启动mongodb服务

```bash
cd /opt/mongodb/bin
./mongod -f mongodb.cfg
```

#### 进入mongodb控制台

```
./mongo
> db.version()
3.6.4
```

如果能打印版本号,则mongodb安装好了,否则会在打印错误信息.
默认情况下，连接地址是127.0.0.1:27017，连接的数据库是test数据库，也可以手动指定连接地址和连接的数据库,如：
`mongo 127.0.0.1:27017/admin`

### 配置开机启动(优先推荐方式一)

#### 方式一: 创建service文件

`touch /lib/systemd/system/mongodb.service`

```bash
[Unit]  
Description=mongodb   
After=network.target remote-fs.target nss-lookup.target  

[Service]  
Type=forking  
ExecStart=/opt/mongodb/bin/mongod --config /opt/mongodb/bin/mongodb.cfg 
ExecReload=/bin/kill -s HUP $MAINPID  
ExecStop=/opt/mongodb/bin/mongod --shutdown --config /opt/mongodb/bin/mongodb.cfg
PrivateTmp=true  

[Install]  
WantedBy=multi-user.target  
```

#### 方式二:编辑/etc/rc.d/rc.local文件

在/etc/rc.d/rc.local文件最后添加:
`/opt/mongodb/bin/mongod --config /opt/mongodb/bin/mongodb.cfg`

### 常见mongodb控制台命令

1. 关闭mongodb服务,注意:这个命令的执行要在admin数据库下，所以先切换到admin，再关闭服务
db.shutdownServer()
2. 退出mongodb控制台
exit

### 创建数据库,账户和密码

**注意:**
1. MongoDB中的账号是在某一个库里面进行设置的，也就是帐号密码和具体的数据库绑定,在哪一个库里进行设置，就要在哪一个库里进行验证。
2. 创建用户时，需要指定用户名、用户密码和用户角色，用户角色表示了该用户的权限。

#### 创建管理员账户和密码

**说明:**
1. 用户管理员是第一个要创建的用户。在没有创建任何用户之前，你可以随意创建用户；但数据库中一旦有了用户，那么未登录的客户端就没有权限做任何操作了，除非使用db.auth(username, password)方法登录。
2. 用户管理员的角色名叫 userAdminAnyDatabase，这个角色只能在 admin 数据库中创建。


#### 创建普通账户和密码

**说明:**
1. 首先保证你已经以用户管理员的身份登录 admin 数据库。然后用 use 命令切换到目标数据库，
2. 用 db.createUser() 命令来创建用户，其中角色名为 “readWrite”。
3. 普通的数据库用户角色有两种，read 和 readWrite。前者只能读取数据不能修改，后者可以读取和修改。

### 常见问题

#### 1. 控制台输入:mongo,报如下错误:

这是mongodb的服务没有启动起来, mongodb服务启动需要root权限,启动mongdb服务即可:
```bash
sudo systemctl start mongodb.service
```